In [1]:
# library preparation
import pandas as pd
import numpy as np
import time
import os

In [2]:
# Custom I/O functions
def read_labels(filename):
    """
    Reads the labels.csv file containing info on
    which samples contain SAS or not

    Input: 
        filename: file path to labels.csv
    Output:
        y: a numpy array of labels "Yes" vs "No"
    """

    y = pd.read_csv(filename)
    y = y.sort_values(by="indices").reset_index(drop=True).to_numpy()

    return y

def read_samples(files, filePath):
    """
    Read in the output files from SA.wrapper.r
    and save it as a compressed file using numpy
    
    Input:
        files: an iterable of names that indicate which output
        files to read; len(samples) must be greater than or equal to one
        
        filePath: path to the folder containing all output files
        
    Output:
        a numpy ndarray of all output files concatenated together
    """

    iterator = iter(files)
    missing = []
    # grab the first output data and use it as a baseline
    firstFile = next(iterator)
    fileStart = os.path.join(filePath, firstFile)
    X = pd.read_table(fileStart, header=None).to_numpy()
    rows, columns = X.shape

    # We now concatenate all samples together
    # Note: THIS COULD TAKE A WHILE! (only run once)
    start = time.time()
    for fileName in iterator:
        fullPath = os.path.join(filePath, fileName)
        if not os.path.exists(fullPath):
            missing.append(fileName)
            continue

        sample = pd.read_table(fullPath, header=None).to_numpy()
        X = np.concatenate((X, sample), axis=0)
    end = time.time()
    print('Time taken to concatenate files:', end - start)

    # reshape to samples, rows, columns
    X = X.reshape(-1, rows, columns)

    return X, missing

def clean_data(X, y, threshold):
    """
    Ensure no element is nan or infinity, otherwise scikit-learn 
    will not like that.
    
    If samples with nan or infinity is below a threshold amount of the 
    total data/samples in X, then we drop them
    
    Input: 
        X: a three dimensional numpy array
            + Assume first dimension = # of samples
            + Assume second dimension = # of rows in each sample
            + Assume third dimension = # of columns in each sample
        y: the corresponding label values for X
        
    Output:
        X_clean: X cleaned if mising data is below threshold, otherwise X
        y_clean: y cleaned if missing data is below threshold, otherwise y
        samples_clean: number of samples left after cleaning (max is X.shape[0])
    """

    assert len(X.shape) == 3
    assert y.shape[0] == X.shape[0]
    samples = X.shape[0]
    print("Dected", samples, "samples total")

    has_NAN = np.any(np.isnan(X))
    samples_with_nans = set()
    if has_NAN:
        print("We got NaNs, identifying location of NaNs...")
        indices_nan = np.argwhere(np.isnan(X))
        samples_with_nans.update(set(indices_nan[:, 0]))
        columns_with_nans = set(indices_nan[:, 2])
        print(f"Attributes with NaNs are: {columns_with_nans}")
        # print(f"Samples with NaNs are: {samples_with_nans}")
        print(f"There are ", len(samples_with_nans), " Samples with NANs")

    has_Inf = np.any(np.isinf(X))
    samples_with_Infs = set()
    if has_Inf:
        print("We got Infinities, identifying location of Infs...")
        indices_Infs = np.argwhere(np.isinf(X))
        samples_with_Infs.update(set(indices_Infs[:, 0]))
        columns_with_Infs = set(indices_Infs[:, 2])
        print(f"Attributes with Infs are: {columns_with_Infs}")
        # print(f"Samples with Infs are: {samples_with_nans}")
        print(f"There are ", len(samples_with_Infs), " Samples with Infs")

    # Attempting to clean the dataset from NaNs and Infs
    bug_samples = set()
    bug_samples.update(samples_with_nans)
    bug_samples.update(samples_with_Infs)
    print(f"There are a total of {len(bug_samples)} containing Infs or NaNs")
    bug_sample_l = list(bug_samples)
    bug_sample_l.sort()
    print("Deleting samples ", bug_sample_l)

    if len(bug_samples) < threshold*samples:
        print(f"Less than {threshold*100}% of Samples, dropping...")
        X_clean = np.delete(X, bug_sample_l, axis = 0)
        y_clean = np.delete(y, bug_sample_l, axis = 0)
        samples_clean = samples - len(bug_sample_l)
        print(f"After dropping, we have {samples_clean} samples left")
    else:
        print("Too many Samples to drop, can't clean!")
        
    return X_clean, y_clean, samples_clean

## Training Data

We first read in training samples using an iterable of filenames and 'compress' down to one dimesnion using `reshape` so that RandomForest can evaluate it correctly

The training samples consist of a 50-50 split betwee neutral samples and SAS samples of shape (160, 2) per sample. 
I choose the 50-50 ratio because **class imabalnce** is often a big headache in model fitting and for the sake of training, a 50-50 split will best enable the classifier to capture the signals of SAS. 

The attribute key is as belows:

1. Maximum fst in a 2.5 rho window
2. Mean-squared error for the highest fst peak in a window

`y` will contain label information for if sample i has SAS or not of shape (# of samples, 1) with one aitribute:

1. SAS (Yes or No)

In [3]:
y_train = read_labels("./GenInput/input/label.csv")
print(y_train[:5])
print(y_train.shape)

[[1 'Yes']
 [2 'Yes']
 [3 'Yes']
 [4 'No']
 [5 'No']]
(10000, 2)


In [4]:
# generator, actual cost = O(1)
files = ("output_" + str(i) for i in range(1, 10000+1))
filePath = "./GenInput/output"

X_train, missing = read_samples(files, filePath)
print(missing)

missing_indices = [int(m[7:]) for m in missing]
if len(missing_indices):
    y_train = np.delete(y_train, missing_indices, axis=0)
print(y_train.shape)
print(X_train.shape)
X_train[0][:10, ] # check first 10 rows of first sample

Time taken to concatenate files: 21.189603328704834
[]
(10000, 2)
(10000, 160, 2)


array([[0.39257336, 0.14608226],
       [0.23921614, 0.0362196 ],
       [0.39257336, 0.1809441 ],
       [0.35295858, 0.15712493],
       [0.35295858, 0.19305147],
       [0.24498803, 0.05261772],
       [0.24498803, 0.0496599 ],
       [0.24498803, 0.0707043 ],
       [0.24498803, 0.05170502],
       [0.24498803, 0.06001001]])

In [5]:
X_clean, y_clean, samples_clean = clean_data(X_train, y_train, threshold=0.01)

toDrop = []
for i in range(0, X_clean.shape[0]):
    sample = X_clean[i, :]
    # 1. Fst must be between 0 to 1.0
    minFst, maxFst = min(sample[:, 0]), max(sample[:, 1])
    # 2. MSE must be between 0 to 1.0 
    minMSE, maxMSE = min(sample[:, 1]), max(sample[:, 1])
    # 3. the mean squared error should almost always be smaller than the maximum Fst 
    if (minFst < 0 or maxFst > 1.0 or 
        minMSE < 0 or minFst > 1.0 or 
        sum(sample[:, 1] <= maxMSE) != sample.shape[0]
       ):
        toDrop.append(i)
print(len(toDrop), "samples did not meet expectations")
X_clean, y_clean = np.delete(X_clean, toDrop, axis=0), np.delete(y_clean, toDrop, axis=0)
print(X_clean.shape)
print(y_clean.shape)

print(X_clean[0, :, :])
print(y_clean[:20, :])

Dected 10000 samples total
There are a total of 0 containing Infs or NaNs
Deleting samples  []
Less than 1.0% of Samples, dropping...
After dropping, we have 10000 samples left
2626 samples did not meet expectations
(7374, 160, 2)
(7374, 2)
[[0.19143744 0.03973952]
 [0.23921614 0.04675881]
 [0.23921614 0.07185455]
 [0.23921614 0.09208617]
 [0.18027272 0.04922432]
 [0.18027272 0.06475553]
 [0.23921614 0.10787962]
 [0.23921614 0.07205316]
 [0.23921614 0.07087022]
 [0.23921614 0.09739757]
 [0.41272825 0.41429405]
 [0.43028743 0.48100667]
 [0.43028743 0.48729017]
 [0.43028743 0.51805551]
 [0.43028743 0.46599337]
 [0.35548305 0.30120695]
 [0.19143744 0.03796099]
 [0.19143744 0.03513313]
 [0.19143744 0.03064425]
 [0.19143744 0.0649706 ]
 [0.19143744 0.07321033]
 [0.19143744 0.06874646]
 [0.23921614 0.09006451]
 [0.30728549 0.16032256]
 [0.30728549 0.24950303]
 [0.23575881 0.08410037]
 [0.23575881 0.07868783]
 [0.23575881 0.09500625]
 [0.30728549 0.16419137]
 [0.30728549 0.16225767]
 [0.30728

In [6]:
# store the result into a numpy object
np.savez_compressed('./data/train.npz', 
                    X_train=X_clean, y_train=y_clean)

## Small Test Data Batch

I then asked the simulation for a 50-50 split, 1000 sample test set that's not used in the model building process at all for an earnest attempt at out-of-sample prediction error.

In [7]:
y_test = read_labels("./GenInputTest/input/label.csv")
print(y_test[:5])
print(y_test.shape)

[[1 'Yes']
 [2 'No']
 [3 'Yes']
 [4 'Yes']
 [5 'Yes']]
(1000, 2)


In [8]:
# generator, actual cost = O(1)
files = ("output_" + str(i) for i in range(1, 1000+1))
filePath = "./GenInputTest/output"

X_test, missing = read_samples(files, filePath)
print(missing)

missing_indices = [int(m[7:]) for m in missing]
if len(missing_indices):
    y_test = np.delete(y_test, missing_indices, axis=0)
print(y_test.shape)
print(y_test[:20, :])
print(X_test.shape)

X_test[0][:10, ] # check first 10 rows of first sample

Time taken to concatenate files: 0.7271561622619629
[]
(1000, 2)
[[1 'Yes']
 [2 'No']
 [3 'Yes']
 [4 'Yes']
 [5 'Yes']
 [6 'Yes']
 [7 'No']
 [8 'No']
 [9 'No']
 [10 'Yes']
 [11 'Yes']
 [12 'Yes']
 [13 'Yes']
 [14 'Yes']
 [15 'No']
 [16 'No']
 [17 'Yes']
 [18 'Yes']
 [19 'Yes']
 [20 'No']]
(1000, 160, 2)


array([[0.43241981, 0.26219987],
       [0.43241981, 0.32465513],
       [0.35295858, 0.24039368],
       [0.43241981, 0.31260821],
       [0.43241981, 0.3246583 ],
       [0.43241981, 0.39181164],
       [0.43241981, 0.40586182],
       [0.43241981, 0.39830596],
       [0.43241981, 0.43595325],
       [0.43241981, 0.37849837]])

In [9]:
X_test_clean, y_test_clean, samples_test = clean_data(
    X_test, y_test, threshold=0.01)
print("\n")
print(X_test_clean.shape)
print(y_test_clean.shape)

toDrop = []
for i in range(0, X_test_clean.shape[0]):
    sample = X_test_clean[i, :]
    # 1. Fst must be between 0 to 1.0
    minFst, maxFst = min(sample[:, 0]), max(sample[:, 1])
    # 2. MSE must be between 0 to 1.0 
    minMSE, maxMSE = min(sample[:, 1]), max(sample[:, 1])
    # 3. the mean squared error should almost always be smaller than the maximum Fst 
    if (minFst < 0 or maxFst > 1.0 or 
        minMSE < 0 or minFst > 1.0 or 
        sum(sample[:, 1] <= maxMSE) != sample.shape[0]
       ):
        toDrop.append(i)
print(len(toDrop), "samples did not meet expectations")
X_test_clean, y_test_clean = np.delete(X_test_clean, toDrop, axis=0), np.delete(y_test_clean, toDrop, axis=0)
print(X_test_clean.shape)
print(y_test_clean.shape)

print(X_test_clean[0, :, :])
print(y_test_clean[:20, :])

Dected 1000 samples total
There are a total of 0 containing Infs or NaNs
Deleting samples  []
Less than 1.0% of Samples, dropping...
After dropping, we have 1000 samples left


(1000, 160, 2)
(1000, 2)
264 samples did not meet expectations
(736, 160, 2)
(736, 2)
[[0.2549095  0.08516194]
 [0.2549095  0.0837489 ]
 [0.2549095  0.08440095]
 [0.35548305 0.15999598]
 [0.28921628 0.10953766]
 [0.28921628 0.10241462]
 [0.28921628 0.11529185]
 [0.23021936 0.0553937 ]
 [0.23921614 0.0545814 ]
 [0.23921614 0.0509845 ]
 [0.23921614 0.06241661]
 [0.23921614 0.0737488 ]
 [0.10734694 0.00863988]
 [0.19143744 0.04645035]
 [0.22997152 0.04599871]
 [0.19143744 0.04506182]
 [0.19143744 0.03975318]
 [0.19143744 0.04393289]
 [0.28921628 0.11273935]
 [0.22997152 0.05295682]
 [0.14460059 0.02191636]
 [0.14460059 0.02033399]
 [0.2549095  0.0882787 ]
 [0.23921614 0.06258111]
 [0.23921614 0.06837051]
 [0.23921614 0.07826347]
 [0.23921614 0.06478117]
 [0.2549095  0.07740275]
 [0.34910714 0.13749933]
 [0.48341837

In [10]:
# store the result into a numpy object
np.savez_compressed('./data/test.npz', X_test=X_test_clean, y_test=y_test_clean)

# New Wrapper Training Samples

In [11]:
y_trainNew = read_labels("./GenInput-New/input/label.csv")
print(y_trainNew[:20])
print(y_trainNew.shape)

[[1 'Yes']
 [2 'Yes']
 [3 'No']
 [4 'Yes']
 [5 'Yes']
 [6 'Yes']
 [7 'No']
 [8 'Yes']
 [9 'Yes']
 [10 'No']
 [11 'Yes']
 [12 'No']
 [13 'No']
 [14 'No']
 [15 'No']
 [16 'No']
 [17 'Yes']
 [18 'Yes']
 [19 'No']
 [20 'No']]
(10000, 2)


In [12]:
# generator, actual cost = O(1)
files = ("output_" + str(i) for i in range(1, 10000+1))
filePath = "./GenInput-New/output"

X_trainNew, missing = read_samples(files, filePath)
print(missing)

missing_indices = [int(m[7:]) for m in missing]
if len(missing_indices):
    y_trainNew = np.delete(y_trainNew, missing_indices, axis=0)
print(y_trainNew.shape)
print(X_trainNew.shape)
print(y_trainNew[:20])
X_trainNew[0][:10, ] # check first 10 rows of first sample

Time taken to concatenate files: 17.11920404434204
['output_3829', 'output_5012', 'output_6767', 'output_7500']
(9996, 2)
(9996, 160, 2)
[[1 'Yes']
 [2 'Yes']
 [3 'No']
 [4 'Yes']
 [5 'Yes']
 [6 'Yes']
 [7 'No']
 [8 'Yes']
 [9 'Yes']
 [10 'No']
 [11 'Yes']
 [12 'No']
 [13 'No']
 [14 'No']
 [15 'No']
 [16 'No']
 [17 'Yes']
 [18 'Yes']
 [19 'No']
 [20 'No']]


array([[0.75204082, 0.45297954],
       [0.75204082, 0.47820333],
       [0.6513074 , 0.44110209],
       [0.60207931, 0.34496555],
       [0.75204082, 0.57374648],
       [0.61255874, 0.29727373],
       [0.72495782, 0.44813823],
       [0.72495782, 0.44213076],
       [0.72495782, 0.46751836],
       [0.72495782, 0.42472071]])

In [13]:
# clean data of Inf and Nans
X_cleanNew, y_cleanNew, samples_clean = clean_data(X_trainNew, y_trainNew, threshold=0.01)
# clean data that does NOT meet expectations
toDrop = []
for i in range(0, X_cleanNew.shape[0]):
    sample = X_cleanNew[i, :]
    # 1. Fst must be between 0 to 1.0
    minFst, maxFst = min(sample[:, 0]), max(sample[:, 1])
    # 2. MSE must be between 0 to 1.0 
    minMSE, maxMSE = min(sample[:, 1]), max(sample[:, 1])
    # 3. the mean squared error should almost always be smaller than the maximum Fst 
    if (minFst < 0 or maxFst > 1.0 or 
        minMSE < 0 or minFst > 1.0 or 
        sum(sample[:, 1] <= maxMSE) != sample.shape[0]
       ):
        toDrop.append(i)
print(len(toDrop), "samples did not meet expectations")
X_cleanNew, y_cleanNew = np.delete(X_cleanNew, toDrop, axis=0), np.delete(y_cleanNew, toDrop, axis=0)
print(X_cleanNew.shape)
print(y_cleanNew.shape)

print(X_cleanNew[0, :, :])
print(y_cleanNew[:20, :])

Dected 9996 samples total
There are a total of 0 containing Infs or NaNs
Deleting samples  []
Less than 1.0% of Samples, dropping...
After dropping, we have 9996 samples left
234 samples did not meet expectations
(9762, 160, 2)
(9762, 2)
[[0.75204082 0.45297954]
 [0.75204082 0.47820333]
 [0.6513074  0.44110209]
 [0.60207931 0.34496555]
 [0.75204082 0.57374648]
 [0.61255874 0.29727373]
 [0.72495782 0.44813823]
 [0.72495782 0.44213076]
 [0.72495782 0.46751836]
 [0.72495782 0.42472071]
 [0.72495782 0.45195588]
 [0.49499908 0.22298095]
 [0.39257336 0.11362164]
 [0.6193874  0.28290537]
 [0.6193874  0.29256814]
 [0.6193874  0.31693046]
 [0.6193874  0.26537553]
 [0.6193874  0.32437042]
 [0.6193874  0.27831314]
 [0.6193874  0.33846372]
 [0.6193874  0.2612371 ]
 [0.6193874  0.29883278]
 [0.6193874  0.29729131]
 [0.6193874  0.42116566]
 [0.6193874  0.37249489]
 [0.6193874  0.35469922]
 [0.6193874  0.36377075]
 [0.6193874  0.36798763]
 [0.39257336 0.1849375 ]
 [0.6193874  0.3275587 ]
 [0.6193874 

In [14]:
# store the result into a numpy object
np.savez_compressed('./data/trainNew.npz', 
                    X_train=X_cleanNew, y_train=y_cleanNew)